# Step 4: Rebalnacing Plan 

## 4.1 Processing Data

### 4.1.1 Get the result of bicycles balance

#### Input：

    od_hourly_zip_poisson_daytype_1.csv
       
#### Output：

    station_daily_balance_daytype_1.csv

In [ ]:
import pandas as pd

# Read demand prediction file
df = pd.read_csv("od_hourly_zip_poisson_daytype_1.csv")
demand_col = "demand_zip_poisson"

# Aggregate departures
borrow = (
    df.groupby("start_station_id")[demand_col]
    .sum()
    .rename("borrowed")
)

# Aggregate returns
returned = (
    df.groupby("end_station_id")[demand_col]
    .sum()
    .rename("returned")
)

# Merge into station-level table
station_balance = pd.concat([borrow, returned], axis=1).fillna(0)

# Compute net flow
station_balance["net"] = station_balance["returned"] - station_balance["borrowed"]

# Define move_in / move_out
station_balance["move_out"] = station_balance["net"].apply(lambda x: x if x > 0 else 0)
station_balance["move_in"] = station_balance["net"].apply(lambda x: -x if x < 0 else 0)

# Reset index to obtain station_id column
station_balance = station_balance.reset_index().rename(columns={"index":"station_id"})

# Save results
station_balance.to_csv("station_daily_balance_daytype_1.csv", index=False)

station_balance.head()

### 4.1.2 Combine data for caulating distance

#### Input：

    station_daily_balance_daytype_1.csv 
    merged_2018_2021.csv      
#### Output：

    station_tasks.csv

 Merge station daily net flows, coordinates, and capacity into a single task table.
 
 Read daily net demand, join with station locations and capacity plan, filter invalid coordinates, and export a cleaned per-station dataset for visualization or further analysis.


In [ ]:
import pandas as pd

# Read daily rebalancing results
balance = pd.read_csv("station_daily_balance_daytype_1.csv")
# Keep only the required columns
balance = balance[["station_id", "move_out", "net","move_in"]]


# Extract station coordinates 
df = pd.read_csv("merged_2018_2021.csv")

# Extract start_station_id and its latitude/longitude
loc = df[["start_station_id", "start_station_latitude", "start_station_longitude"]].drop_duplicates()

# Rename columns 
loc = loc.rename(columns={
    "start_station_id": "station_id",
    "start_station_latitude": "latitude",
    "start_station_longitude": "longitude"
})

#  Extract initial stock x_i and capacity y_i from Minlimit_capacity_daytype_1.csv
cap = pd.read_csv("Minlimit_capacity_daytype_1.csv")

cap = cap.rename(columns={
    "x_i": "initial_stock",
    "y_i": "capacity"
})

# Merge the three tables

# Merge balance with location
df_merge = balance.merge(loc, on="station_id", how="left")

# Merge with capacity info
df_merge = df_merge.merge(cap[["station_id", "initial_stock", "capacity"]],
                          on="station_id", how="left")


#Remove stations with missing coordinates

# Latitude/longitude are left-joined from loc; missing ones are NaN
print(f"Station count before dropping: {len(df_merge)}")

# Drop rows with latitude or longitude is NaN
df_merge.dropna(subset=["latitude", "longitude"], inplace=True)

# Convert to numeric types, drop invalid values 
df_merge["latitude"] = pd.to_numeric(df_merge["latitude"], errors='coerce')
df_merge["longitude"] = pd.to_numeric(df_merge["longitude"], errors='coerce')
df_merge.dropna(subset=["latitude", "longitude"], inplace=True)

print(f"Station count after dropping: {len(df_merge)}")




# Remove duplicate stations
df_merge = df_merge.drop_duplicates(subset=["station_id"], keep="first")


# Reorder columns
df_merge = df_merge[[
    "station_id",
    "latitude",
    "longitude",
    "initial_stock",
    "capacity",
    "net",
    "move_out",
    "move_in"
]]

# Export the final station task file
df_merge.to_csv("station_tasks.csv", index=False)

df_merge.head()